# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through the Universal Dependencies (https://universaldependencies.org/) (UD) project. To get to know the project, please visit https://universaldependencies.org/introduction.html)

In [ ]:
import numpy as np
import operator
import nltk
from statistics import mode
import pandas as pd
import itertools
import random

!pip install conllutils
import conllutils

  Created wheel for conllutils: filename=conllutils-1.1.4-cp37-none-any.whl size=17696 sha256=9d25de27e8310cd0228b657245a722a41125fab690c9bf6739b918cdbf9e7c1b
  Stored in directory: /root/.cache/pip/wheels/94/08/3a/74fab1fdde78b548f26b2de930af3cee81a592fafe9e758a57
Successfully built conllutils


**Part 1** (getting the data)

You can download the dataset files directly from the UD website, but it will let you only download all the languages in one compressed file. In this assignment you will be working with th GUM dataset, which you can download directly from:
https://github.com/UniversalDependencies/UD_English-GUM.
Please download it to your colab machine.



In [ ]:
!git clone https://github.com/UniversalDependencies/UD_English-GUM

Cloning into 'UD_English-GUM'...
remote: Enumerating objects: 2518, done.
remote: Counting objects: 100% (1529/1529), done.
remote: Compressing objects: 100% (660/660), done.
remote: Total 2518 (delta 1342), reused 1050 (delta 869), pack-reused 989
Receiving objects: 100% (2518/2518), 16.21 MiB | 9.06 MiB/s, done.
Resolving deltas: 100% (2232/2232), done.


We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

In [ ]:
train = conllutils.read_conllu('UD_English-GUM/en_gum-ud-train.conllu')
dev = conllutils.read_conllu('UD_English-GUM/en_gum-ud-dev.conllu')
test = conllutils.read_conllu('UD_English-GUM/en_gum-ud-test.conllu')

In [ ]:
train_data = []
for sentence in train:
  train_sentence = []
  for token in sentence:
    train_sentence.append((token.form, token.upos))
  train_data.append(train_sentence)

dev_data = []
for sentence in dev:
  dev_sentence = []
  for token in sentence:
    dev_sentence.append((token.form, token.upos))
  dev_data.append(dev_sentence)

test_data = []
for sentence in test:
  test_sentence = []
  for token in sentence:
    test_sentence.append((token.form, token.upos))
  test_data.append(test_sentence)

In [ ]:
print(train_data)
print(dev_data)
print(test_data)

[[('Aesthetic', 'ADJ'), ('Appreciation', 'NOUN'), ('and', 'CCONJ'), ('Spanish', 'ADJ'), ('Art', 'NOUN'), (':', 'PUNCT')], [('Insights', 'NOUN'), ('from', 'ADP'), ('Eye-Tracking', 'NOUN')], [('Claire', 'PROPN'), ('Bailey-Ross', 'PROPN'), ('claire.bailey-ross@port.ac.uk', 'PROPN'), ('University', 'PROPN'), ('of', 'ADP'), ('Portsmouth', 'PROPN'), (',', 'PUNCT'), ('United', 'PROPN'), ('Kingdom', 'PROPN')], [('Andrew', 'PROPN'), ('Beresford', 'PROPN'), ('a.m.beresford@durham.ac.uk', 'PROPN'), ('Durham', 'PROPN'), ('University', 'PROPN'), (',', 'PUNCT'), ('United', 'PROPN'), ('Kingdom', 'PROPN')], [('Daniel', 'PROPN'), ('Smith', 'PROPN'), ('daniel.smith2@durham.ac.uk', 'PROPN'), ('Durham', 'PROPN'), ('University', 'PROPN'), (',', 'PUNCT'), ('United', 'PROPN'), ('Kingdom', 'PROPN')], [('Claire', 'PROPN'), ('Warwick', 'PROPN'), ('c.l.h.warwick@durham.ac.uk', 'PROPN'), ('Durham', 'PROPN'), ('University', 'PROPN'), (',', 'PUNCT'), ('United', 'PROPN'), ('Kingdom', 'PROPN')], [('How', 'ADV'), ('do

**Part 2**

Write a class **simple_tagger**, with methods *train* and *evaluate*. The method *train* receives the data as a list of sentences, and use it for training the tagger. In this case, it should learn a simple dictionary that maps words to tags, defined as the most frequent tag for every word (in case there is more than one most frequent tag, you may select one of them randomly). The dictionary should be stored as a class member for evaluation.

The method *evaluate* receives the data as a list of sentences, and use it to evaluate the tagger performance. Specifically, you should calculate the word and sentence level accuracy.
The evaluation process is simply going word by word, querying the dictionary (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set (i.e., the mode). The function should return the two numbers: word level accuracy and sentence level accuracy.


In [ ]:
class simple_tagger:
  def train(self, data):
    all_tokens = list(set(item[0] for temp in data for item in temp)) # list of all unique tokens
    all_pos = list(set(item[1] for temp in data for item in temp)) # list of all unique parts of speech
    
    counter = np.zeros((len(all_tokens), len(all_pos)))
    for sentence in data:
      for item in sentence:
        counter[all_tokens.index(item[0]), all_pos.index(item[1])] = counter[all_tokens.index(item[0]), all_pos.index(item[1])] + 1
    
    max_indices = np.argmax(counter, axis=1) # find most frequent pos for each token
    max_pos = [all_pos[index] for index in max_indices] # convert pos' index to its name
    
    self.tags = dict(zip(all_tokens, max_pos)) # create tags dictionary
    self.most_freq_tag = mode(max_pos) # for OOV words

  
  def evaluate(self, data):
    num_of_sentences = len(data)
    num_of_tokens = len([item for sublist in data for item in sublist])
    correct_sentences = 0.0
    correct_tokens = 0.0
    
    for sentence in data:
      flag = True # for sentence level accuracy
      for item in sentence:
        if item[1] == self.tags.setdefault(item[0], self.most_freq_tag):
          correct_tokens = correct_tokens + 1
        else:
          flag = False # if one token in the sentence is tagged wrong, the whole sentence is tagged wrong
      if flag:
        correct_sentences = correct_sentences + 1

    word_level_acc = correct_tokens / num_of_tokens
    sentence_level_acc = correct_sentences / num_of_sentences

    return word_level_acc, sentence_level_acc

**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence using he Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. You should implement the Viterbi algorithm in the next block and call it from your class.

Additional guidance:
1. The matrix B represents the emissions probabilities. Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way columns in B represents word ids.
2. During the evaluation, you should first convert each word into it’s index and then create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see an OOV word, and decode every segment individually using Viterbi.


In [ ]:
class hmm_tagger:
  def A_helper(self, pos1, pos2, data):
    # returns the conditional probability P(pos2|pos1)
    poss = [item[1] for sublist in data for item in sublist]
    count_pos1 = len([p for p in poss if p == pos1]) # number of occurrences of pos1 in the data
    count_pos2_pos1 = 0.0
    for i in range(len(poss)-1):
        if poss[i] == pos1 and poss[i+1] == pos2:
          count_pos2_pos1 = count_pos2_pos1 + 1
    return count_pos2_pos1 / count_pos1

  def B_helper(self, token, pos, data):
    # returns the conditional probability P(token|pos)
    poss = [item for sublist in data for item in sublist if item[1] == pos]
    count_pos = len(poss) # number of occurrences of pos in the data
    token_pos = [item[0] for item in poss if item[0] == token] 
    count_token_pos = float(len(token_pos)) # number of occurrences of (token, pos) in the data
    return count_token_pos / count_pos

  def Pi_helper(self, pos, data):
    # returns the initial probability P(pos)
    poss = [item for sublist in data for item in sublist if item[1] == pos]
    count_pos = float(len(poss)) # number of occurrences of pos in the data
    return count_pos / len([item for sublist in data for item in sublist])

  def sentence_to_ids(self, sentence):
    # convert a sentence to a sequence of ids
    tokens = [item[0] for item in sentence]
    ids = []
    for token in tokens:
      ids.append(self.token_id.setdefault(token, -1)) # OOV words' id is -1
    return ids

  def random_pos(self):
    # returns a random pos for OOV words' tagging
    return self.all_pos[random.randint(0, len(self.all_pos))]


  def train(self, data):
    all_tokens = list(set(item[0] for temp in data for item in temp)) # list of all unique tokens
    self.token_id = dict(zip(all_tokens, list(range(0, len(all_tokens))))) # a dictionary that maps every unique word in the corpus to a serial numeric id
    all_pos = list(set(item[1] for temp in data for item in temp)) # list of all unique parts of speech
    self.all_pos = all_pos # for later use (in evaluate)

    # create A - transition matrix
    self.A = np.empty((len(all_pos), len(all_pos)))
    for i, pos1 in enumerate(all_pos):
      for j, pos2 in enumerate(all_pos): 
        self.A[i, j] = self.A_helper(pos1, pos2, data)

    # create B - emission matrix
    self.B = np.empty((len(all_pos), len(all_tokens)))
    for i, pos in enumerate(all_pos):
      for j, token in enumerate(all_tokens): 
        self.B[i, j] = self.B_helper(token, pos, data)

    # create Pi - initial probabilities vector
    self.Pi = np.empty(len(all_pos))
    for i, pos in enumerate(all_pos):
      self.Pi[i] = self.Pi_helper(pos, data)


  def evaluate(self, data):
    num_of_sentences = len(data)
    num_of_tokens = len([item for sublist in data for item in sublist])
    correct_sentences = 0.0
    correct_tokens = 0.0
    
    for sentence in data:
      flag = True # for sentence level accuracy
      observation = self.sentence_to_ids(sentence)
      best_sequence = np.empty(len(observation), dtype=int)
      
      # break the sentence into multiple segments every time we see an OOV word, and decode every segment individually using Viterbi
      prev_OOV = 0
      for i in range(0,len(observation)):
        if i == -1:
          if prev_OOV != i:
            best_sequence[prev_OOV:i] = viterbi(observation[prev_OOV:i], self.A, self.B, self.Pi) # 
          best_sequence[i] = self.random_pos() # assign OOV words with random tag
          prev_OOV = i+1
      best_sequence[prev_OOV:len(observation)] = viterbi(observation[prev_OOV:len(observation)], self.A, self.B, self.Pi)
      
      predicted = [self.all_pos[i] for i in best_sequence] # convert a sequence of ids to a sequence of pos
      
      for i,item in enumerate(sentence):
        if item[1] == predicted[i]:
          correct_tokens = correct_tokens + 1
        else:
          flag = False # if one token in the sentence is tagged wrong, the whole sentence is tagged wrong
      if flag:
        correct_sentences = correct_sentences + 1

    word_level_acc = correct_tokens / num_of_tokens
    sentence_level_acc = correct_sentences / num_of_sentences

    return word_level_acc, sentence_level_acc

In [ ]:
# Viterbi
def viterbi (observations, A, B, Pi):
  T = len(observations)
  delta = np.empty((A.shape[0], T))
  psi = np.empty((A.shape[0], T))

  # initialization step
  delta[:, 0] = Pi * B[:, observations[0]]
  psi[:, 0] = 0
  
  # iteration step
  for t in range(1, T):
    delta[:, t] = B[:, observations[t]] * np.max(delta[:, t-1] * A.T ,1)
    psi[:, t] = np.argmax(delta[:, t-1] * A.T, 1)

  # sequence recovery
  best_sequence = np.empty(T, dtype=int)
  best_sequence[T-1] = np.argmax(delta[:, T-1])
  for t in reversed(range(1, T)):
    best_sequence[t-1] = psi[best_sequence[t], t]

  return best_sequence

Viterbi's validation:

In [ ]:
A = np.array([[0.3, 0.7], [0.2, 0.8]])
B = np.array([[0.1, 0.1, 0.3, 0.5], [0.3, 0.3, 0.2, 0.2]])
Pi = np.array([0.4, 0.6])
print(viterbi([0, 3, 2, 0], A, B, Pi))
# Expected output: 1, 1, 1, 1

[1 1 1 1]


**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK (a known NLP library), over the test data. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [ ]:
from nltk.tag import tnt 

tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_data)

In [ ]:
simple = simple_tagger()
simple.train(train_data)

In [ ]:
hmm = hmm_tagger()
hmm.train(train_data)

Print both, word level and sentence level accuracy for all the three taggers in a table.

In [ ]:
acc = np.zeros((3,4)) 

# evaluate taggers based on dev dataset
acc[0,:2] = simple.evaluate(dev_data)
acc[1,:2] = hmm.evaluate(dev_data)
acc[2,0] = tnt_pos_tagger.evaluate(dev_data) # word level accuracy
# calculate sentence level accuracy of MEMM
correct_sentences = 0.0
for sentence in dev_data:
  if tnt_pos_tagger.evaluate([sentence]) == 1.0: # all the tokens in the sentence are tagged correctly
    correct_sentences = correct_sentences + 1
acc[2,1] = correct_sentences / len(dev_data)

# evaluate taggers based on test dataset
acc[0,2:] = simple.evaluate(test_data)
acc[1,2:] = hmm.evaluate(test_data)
acc[2,2] = tnt_pos_tagger.evaluate(test_data) # word level accuracy
# calculate sentence level accuracy of MEMM
correct_sentences = 0.0
for sentence in test_data:
  if tnt_pos_tagger.evaluate([sentence]) == 1.0: # all the tokens in the sentence are tagged correctly
    correct_sentences = correct_sentences + 1
acc[2,3] = correct_sentences / len(test_data)

print("Taggers' Accuracies")
acc_df = pd.DataFrame(data=acc, index=['simple', 'HMM', 'MEMM'], columns=['word - dev', 'sentence - dev', 'word - test', 'sentence - test'])
print(acc_df)

Taggers' Accuracies
        word - dev  sentence - dev  word - test  sentence - test
simple    0.856648        0.161990     0.846164         0.189888
HMM       0.873061        0.198980     0.860919         0.225843
MEMM      0.827863        0.127551     0.802336         0.123596
